In [1]:
!pip install array-to-latex

  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1850 sha256=05060c748807cb5b517b1eb2cf2a15296e255dd158adc5dc54d8782103701fca
  Stored in directory: c:\users\sambi\appdata\local\pip\cache\wheels\58\b3\11\d2a638f07f2b7f9210a1a1e8fd621005a836fa1dbf426a1553
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=4260d2bfaaa4fb9a3c564ab4341a56bb7b403ca22bac302e3333af03949adfc2
  Stored in directory: c:\users\sambi\appdata\local\pip\cache\wheels\0c\09\9e\49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built clipboard pyperclip


In [154]:
def to_scientific_notation(number):
    a, b = '{:.1e}'.format(number).split('e')
    return '{:.2f}e{:+03d}'.format(float(a)/10, int(b)+1)

to_scientific_notation(113.2)

'0.11e+03'

In [165]:
import numpy as _np
import pandas as _pd

def _numpyarraytolatex(a, arraytype='bmatrix', nargout=0,
                       imstring='j', row=True, mathform=True):
    r"""Return a LaTeX array given a numpy array.

    Parameters
    ----------
    a         : float array
    frmt      : string
        python 3 formatter, optional-
        https://mkaz.tech/python-string-format.html
    arraytype : string
        latex array type- `bmatrix` default, optional
    imstring : string (optional)
        Character for square root of -1. Usually i or j
    row      : Boolean
        If the array is 1-D, should the output be
            a row (True) or column (False)

    Returns
    -------
    out: str
        LaTeX array

    See Also
    --------
    to_clp

    Examples
    --------
    >>> import numpy as np
    >>> import array_to_latex as a2l
    >>> A = np.array([[1.23456, 23.45678],[456.23, 8.239521]])
    >>> a2l.to_ltx(A, frmt = '{:6.2f}', arraytype = 'array')
    \begin{array}{c, c}
        1.23 &   23.46\\
      456.23 &    8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:6.2e}', arraytype = 'array')
    \begin{array}{c, c}
      1.23e+00 &  2.35e+01\\
      4.56e+02 &  8.24e+00
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:.3g}', arraytype = 'array')
    \begin{array}{c, c}
      1.23 &  23.5\\
      456 &  8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:1.2f}', arraytype = 'coords')
    {(1.23,23.46),(456.23,8.24)}
    None

    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')

    if len(a.shape) == 1:
        a = _np.array([a])
        if row is False:
            a = a.T

    if arraytype == "coords":
        coords = ['(' + ','.join([to_scientific_notation(x) for x in r]) + ')' for r in a]
        return '{' + ','.join(coords) + '}'

    arrayformat = ''

    if arraytype == 'array':
        arrayformat = '{'
        for _ in _np.arange(a.shape[1]):
            arrayformat = arrayformat + ' c,'
        arrayformat = arrayformat[:-1] + '}'

    out = r'\begin{' + arraytype + '}' + arrayformat + '\n'
    for i in _np.arange(a.shape[0]):
        out = out + ' '
        for j in _np.arange(a.shape[1]):
            leadstr = '' if _np.real(a[i, j]) < 0 else ' '
            dot_space = ' ' if '.' not in to_scientific_notation(a[i, j]) else ''
            if _np.iscomplexobj(a[i, j]):
                out = (out + leadstr
                       + math_form(to_scientific_notation(_np.real(a[i, j])),
                                   mathform=mathform)
                       + ' + '
                       + math_form(to_scientific_notation(_np.imag(a[i, j])),
                                   is_imaginary=True,
                                   mathform=mathform)
                       + imstring
                       + dot_space + ' & ')
            else:
                out = (out
                       + leadstr
                       + math_form(to_scientific_notation(_np.real(a[i, j])),
                                   mathform=mathform)
                       + dot_space
                       + r' & ')

        out = out[:-3]
        out = out + '\\\\\n'

    out = out[:-3] + '\n' + r'\end{' + arraytype + '}'

    return out

def to_ltx(a, frmt='{:1.2f}', arraytype=None, nargout=0,
           imstring='j', row=True, mathform=True, print_out=True):
    r"""
    Print or return a LaTeX array given a numpy array or Pandas dataframe.

    Parameters
    ----------
    a         : float array
    frmt      : string
        python 3 formatter, optional-
        https://mkaz.tech/python-string-format.html
    arraytype : string
        latex array type- `bmatrix` default, optional
    imstring : string (optional)
        Character for square root of -1. Usually i or j
    row        : Boolean (optional: default True)
        If the array is 1-D, should the output be
            a row (True) or column (False)
    mathform  : Boolean (optional: default True)
        Replace #E# with #\times10^{#}


    Returns
    -------
    out: str
        LaTeX array

    See Also
    --------
    to_clp

    Examples
    --------
    >>> import numpy as np
    >>> import array_to_latex as a2l
    >>> A = np.array([[1.23456, 23.45678],[456.23, 8.239521]])
    >>> a2l.to_ltx(A, frmt = '{:6.2f}', arraytype = 'array')
    \begin{array}
        1.23 &   23.46\\
      456.23 &    8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:6.2e}', arraytype = 'array')
    \begin{array}
      1.23e+00 &  2.35e+01\\
      4.56e+02 &  8.24e+00
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:.3g}', arraytype = 'array')
    \begin{array}
      1.23 &  23.5\\
      456  &  8.24
    \end{array}
    None
    >>> a2l.to_ltx(A, frmt = '{:1.2f}', arraytype = 'coords')
    {(1.23,23.46),(456.23,8.24)}
    None

    """
    if isinstance(a, _np.ndarray):

        if arraytype is None:
            arraytype = 'bmatrix'
        latex = _numpyarraytolatex(a, arraytype=arraytype,
                                   nargout=nargout, imstring=imstring,
                                   row=row, mathform=mathform)

    elif isinstance(a, _pd.core.frame.DataFrame):

        if arraytype is None:
            arraytype = 'tabular'
        latex = _dataframetolatex(a, frmt=frmt, arraytype=arraytype,
                                  nargout=nargout, imstring=imstring)
    else:
        raise TypeError("Argument should be a "
                        "numpy array or a pandas DataFrame.")
    if print_out is True:
        print(latex)
        return

    return latex

def math_form(number, is_imaginary=False, mathform=True):
    if 'e' in number:
        if mathform:
            number = number.replace('e', '\\times 10^{') + '}'
        else:
            number = number.replace('e', '\\mathrm{e}{') + '}'
    return number

In [166]:
import numpy as np

# Relaxation method
def relaxation_method(A, b, x0, tolerance=0.01, max_iterations=1000, omega=1.25):
    """
    Solve a system of linear equations using the Relaxation method.

    Arguments:
    A: 2D array representing the coefficients of the linear equations
    b: 1D array representing the constants on the right-hand side of the equations
    x0: 1D array representing the initial guess for the solution
    tolerance: Desired tolerance for the solution
    max_iterations: Maximum number of iterations allowed
    omega: Relaxation parameter (typically between 1 and 2)

    Returns:
    solution: 1D array representing the solution to the system of linear equations
    num_iterations: Number of iterations performed
    """
    n = len(b)
    solution = np.copy(x0)
    for iteration in range(max_iterations):
        old_solution = np.copy(solution)
        for i in range(n):
            summation = 0.0
            for j in range(n):
                if j != i:
                    summation=summation+A[i][j] * solution[j]
            solution[i] = (1 - omega) * old_solution[i] + (omega / A[i][i]) * (b[i] - summation)
        if np.linalg.norm(solution - old_solution) < tolerance:
            return solution, iteration + 1
    print("Warning: Maximum number of iterations reached without convergence.")
    return solution, max_iterations

# Solving by Relaxation method
solution, num_iterations = relaxation_method(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", num_iterations)


Solution: [0 0]
Number of iterations: 1


In [167]:
import numpy as np
# Conjugate Gradient method with latex output
def conjugate_gradient(A, b, x0, tol=0.001, max_iter=1000):
    """
    Solve the linear system Ax = b using Conjugate Gradient method.

    Parameters:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        x0 (numpy.ndarray): Initial guess for the solution.
        tol (float): Tolerance for convergence.
        max_iter (int): Maximum number of iterations.

    Returns:
        x (numpy.ndarray): Solution vector.
        num_iter (int): Number of iterations performed.
    """
    np.set_printoptions(formatter={'float_kind':to_scientific_notation})
    r = b - np.dot(A, x0)
    p = r
    #print("p=")
    #a2l.to_ltx(p, frmt = fm, arraytype = 'bmatrix')
    x = x0
    rsold = np.dot(r, r)
    #print("|r|^2=",rsold)
    for i in range(max_iter):
        print("Iteration",i,":")
        print("r=")
        to_ltx(r, arraytype = 'bmatrix')
        Ap = np.dot(A, p)
        print("Ap=")
        to_ltx(Ap, arraytype = 'bmatrix')
        alpha = rsold / np.dot(p, Ap)
        print("alpha=",to_scientific_notation(alpha))
        x = x + alpha * p
        print("x=")
        to_ltx(x, arraytype = 'bmatrix')
        r = r - alpha * Ap
        print("r=")
        to_ltx(r, arraytype = 'bmatrix')
        rsnew = np.dot(r, r)
        print("|r|^2=",to_scientific_notation(rsnew))
        beta = (rsnew / rsold)
        print("beta",to_scientific_notation(beta))
        if np.sqrt(rsnew) < tol:
            break
        p = r + beta * p
        print("new p=")
        to_ltx(p, arraytype = 'bmatrix')
        rsold = rsnew
    return x, i+1

In [168]:
# Solving by Conjugate Gradient method
A = np.array([[1.00,0.50],[0.50,0.33]])
b = np.array([0.24,0.13])
x0 = np.array([0,0])
solution, iterations = conjugate_gradient(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", iterations)

Iteration 0 :
r=
\begin{bmatrix}
  0.24\times 10^{+00} &  0.13\times 10^{+00}
\end{bmatrix}
Ap=
\begin{bmatrix}
  0.30\times 10^{+00} &  0.16\times 10^{+00}
\end{bmatrix}
alpha= 0.79e+00
x=
\begin{bmatrix}
  0.19\times 10^{+00} &  0.10\times 10^{+00}
\end{bmatrix}
r=
\begin{bmatrix}
 -0.76\times 10^{-03} &  0.14\times 10^{-02}
\end{bmatrix}
|r|^2= 0.26e-05
beta 0.34e-04
new p=
\begin{bmatrix}
 -0.75\times 10^{-03} &  0.14\times 10^{-02}
\end{bmatrix}
Iteration 1 :
r=
\begin{bmatrix}
 -0.76\times 10^{-03} &  0.14\times 10^{-02}
\end{bmatrix}
Ap=
\begin{bmatrix}
 -0.48\times 10^{-04} &  0.89\times 10^{-04}
\end{bmatrix}
alpha= 0.16e+02
x=
\begin{bmatrix}
  0.18\times 10^{+00} &  0.13\times 10^{+00}
\end{bmatrix}
r=
\begin{bmatrix}
 -0.17\times 10^{-15} & -0.91\times 10^{-16}
\end{bmatrix}
|r|^2= 0.37e-31
beta 0.15e-25
Solution: [0.18e+00 0.13e+00]
Number of iterations: 2


In [132]:
# Solving by Conjugate Gradient method
A = np.array([[0.1,0.2],[0.2,11e1]])
b = np.array([0.3,11e1])
x0 = np.array([0,0])
solution, iterations = conjugate_gradient(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", iterations)

Iteration 0 :
r=
\begin{bmatrix}
  3.0\times 10^{-01} &  1.1\times 10^{+02}
\end{bmatrix}
Ap=
\begin{bmatrix}
  2.2\times 10^{+01} &  1.2\times 10^{+04}
\end{bmatrix}
alpha= 9.1e-03
x=
\begin{bmatrix}
  2.7\times 10^{-03} &  1.0\times 10^{+00}
\end{bmatrix}
r=
\begin{bmatrix}
  1.0\times 10^{-01} & -2.7\times 10^{-04}
\end{bmatrix}
|r|^2= 9.9e-03
beta 8.2e-07
new p=
\begin{bmatrix}
  1.0\times 10^{-01} & -1.8\times 10^{-04}
\end{bmatrix}
Iteration 1 :
r=
\begin{bmatrix}
  1.0\times 10^{-01} & -2.7\times 10^{-04}
\end{bmatrix}
Ap=
\begin{bmatrix}
  9.9\times 10^{-03} & -2.7\times 10^{-05}
\end{bmatrix}
alpha= 1.0e+01
x=
\begin{bmatrix}
  1.0\times 10^{+00} &  1.0\times 10^{+00}
\end{bmatrix}
r=
\begin{bmatrix}
  3.9\times 10^{-14} &  2.1\times 10^{-11}
\end{bmatrix}
|r|^2= 4.5e-22
beta 4.6e-20
Solution: [1.0e+00 1.0e+00]
Number of iterations: 2


In [118]:
import numpy as np
import array_to_latex as a2l
# Conjugate Gradient method
def conjugate_gradient(A, b, x0, tol=0.001, max_iter=1000):
    """
    Solve the linear system Ax = b using Conjugate Gradient method.

    Parameters:
        A (numpy.ndarray): Coefficient matrix.
        b (numpy.ndarray): Right-hand side vector.
        x0 (numpy.ndarray): Initial guess for the solution.
        tol (float): Tolerance for convergence.
        max_iter (int): Maximum number of iterations.

    Returns:
        x (numpy.ndarray): Solution vector.
        num_iter (int): Number of iterations performed.
    """
    fm = '{:1.1e}'
    float_formatter = fm.format
    np.set_printoptions(formatter={'float_kind':float_formatter})
    r = b - np.dot(A, x0)
    v = r
    x = x0
    rsold = np.dot(r, r)
    for i in range(max_iter):
        print("Iteration",i,":")
        print("r=")
        a2l.to_ltx(r, frmt = fm, arraytype = 'bmatrix')
        Av = np.dot(A, v)
        print("Av=")
        a2l.to_ltx(Av, frmt = fm, arraytype = 'bmatrix')
        t = np.dot(r,v) / np.dot(v, Av)
        print("t=",fm.format(t))
        x = x + t * v
        print("x=")
        a2l.to_ltx(x, frmt = fm, arraytype = 'bmatrix')
        r = b - np.dot(A,x)
        v = r
        print("r=")
        a2l.to_ltx(r, frmt = fm, arraytype = 'bmatrix')
        rsnew = np.dot(r, r)
        print("|r|^2=",fm.format(rsnew))
        if np.sqrt(rsnew) < tol:
            break
    return x, i+1

# Solving by Conjugate Gradient method
A = np.array([[1.00,0.50],[0.50,0.33]])
b = np.array([0.24,0.13])
x0 = np.array([0,0])
solution, iterations = conjugate_gradient(A, b, x0)
print("Solution:", solution)
print("Number of iterations:", iterations)


Iteration 0 :
r=
\begin{bmatrix}
  2.4\times 10^{-01} &  1.3\times 10^{-01}
\end{bmatrix}
Av=
\begin{bmatrix}
  3.0\times 10^{-01} &  1.6\times 10^{-01}
\end{bmatrix}
t= 7.9e-01
x=
\begin{bmatrix}
  1.9\times 10^{-01} &  1.0\times 10^{-01}
\end{bmatrix}
r=
\begin{bmatrix}
 -7.6\times 10^{-04} &  1.4\times 10^{-03}
\end{bmatrix}
|r|^2= 2.6e-06
Iteration 1 :
r=
\begin{bmatrix}
 -7.6\times 10^{-04} &  1.4\times 10^{-03}
\end{bmatrix}
Av=
\begin{bmatrix}
 -5.9\times 10^{-05} &  8.3\times 10^{-05}
\end{bmatrix}
t= 1.6e+01
x=
\begin{bmatrix}
  1.8\times 10^{-01} &  1.2\times 10^{-01}
\end{bmatrix}
r=
\begin{bmatrix}
  1.7\times 10^{-04} &  9.0\times 10^{-05}
\end{bmatrix}
|r|^2= 3.6e-08
Solution: [1.8e-01 1.2e-01]
Number of iterations: 2


In [102]:
np.format_float_scientific(23.2224,exp_digits=2)

'2.32224e+01'